In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
from privacypacking.budget import LaplaceCurve, Budget, GaussianCurve, SubsampledGaussianCurve
from privacypacking.utils.plot import plot_budgets

In [4]:
block = Budget.from_epsilon_delta(epsilon=10, delta=1e-5)

In [7]:
lap1 = LaplaceCurve(laplace_noise=1.0)
lap2 = LaplaceCurve(laplace_noise=0.5)

In [8]:
plot_budgets([block, lap1, lap2])

In [9]:
import yaml

In [8]:
print(yaml.dump(
    {
        "alphas":lap2.alphas,
        "rdp_epsilons": lap2.epsilons,
        "n_blocks": 100
    }
))

alphas:
- 1.5
- 1.75
- 2
- 2.5
- 3
- 4
- 5
- 6
- 8
- 16
- 32
- 64
n_blocks: 100
rdp_epsilons:
- 1.4368091584145997
- 1.528278115013663
- 1.595773500587618
- 1.6867984187182614
- 1.7446023211979131
- 1.8134616119036404
- 1.8530533368204662
- 1.8787728393324281
- 1.9101987629396724
- 1.955906767850309
- 1.9781484250454255
- 1.9891221586809693



In [55]:
g1 = GaussianCurve(sigma=15) + LaplaceCurve(laplace_noise=5.0)
g2 = GaussianCurve(sigma=10) + LaplaceCurve(laplace_noise=10.0)
plot_budgets([block, g1, g2])


In [43]:
def demand_fractions(curve, block):
    demand_fractions = []
    # Compute the demand share for each alpha of the block
    for alpha in g1.alphas:

        # Drop RDP orders that are already negative
        if block.epsilon(alpha) > 0:
            demand_fractions.append(
                curve.epsilon(alpha) / block.epsilon(alpha)
            )
    return demand_fractions

In [62]:
block

Budget({1.5: 0, 1.75: 0, 2: 0, 2.5: 2.3247163566865146, 3: 4.243537267514886, 4: 6.162358178343258, 5: 7.121768633757442, 6: 7.697414907005954, 8: 8.35529636214711, 16: 9.232471635668652, 32: 9.628615307581606, 64: 9.817255151349679})

In [61]:
demand_fractions(g1, block)

[0.022090015196058032,
 0.014356702028801138,
 0.012806219623761638,
 0.01336939547704545,
 0.014260615031538396,
 0.016045891646521426,
 0.020751652414668642,
 0.025887381889228945,
 0.033751224328501216]

In [60]:
demand_fractions(g2, block)

[0.01054841717253714,
 0.0069224825382308685,
 0.006327320456706545,
 0.006803744546768179,
 0.007500833600541227,
 0.009057341613544777,
 0.01501921860465877,
 0.02473935773520301,
 0.041673787636912804]

In [58]:
print(yaml.dump(
    {
        "alphas":g1.alphas,
        "rdp_epsilons": g1.epsilons,
        "n_blocks": 1
    }
))
print(yaml.dump(
    {
        "alphas":g2.alphas,
        "rdp_epsilons": g2.epsilons,
        "n_blocks": 1
    }
))

alphas:
- 1.5
- 1.75
- 2
- 2.5
- 3
- 4
- 5
- 6
- 8
- 16
- 32
- 64
n_blocks: 1
rdp_epsilons:
- 0.0313023396305879
- 0.03640957567004681
- 0.041459381262075753
- 0.05135301964572977
- 0.0609232000978242
- 0.07891651223214745
- 0.0952137413607209
- 0.10976987072683683
- 0.13406818010158716
- 0.19158904231168314
- 0.24925964153184066
- 0.3313443809033372

alphas:
- 1.5
- 1.75
- 2
- 2.5
- 3
- 4
- 5
- 6
- 8
- 16
- 32
- 64
n_blocks: 1
rdp_epsilons:
- 0.01474748538932854
- 0.017197863456887945
- 0.01964420784034461
- 0.024522077938150005
- 0.029375812634703728
- 0.038991214963384174
- 0.048454694505271866
- 0.05773702837177719
- 0.07567677343437432
- 0.138664509757419
- 0.2382057585889131
- 0.4091222063547348



In [64]:
import pandas as pd
from privacypacking.utils.utils import LOGS_PATH
from privacypacking.utils.analysis import load_ray_experiment
import plotly.express as px

In [10]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2021-09-03_19-12-08")
df = load_ray_experiment(logs)

In [11]:
df.describe()

,n_allocated_tasks,realized_profit,n_tasks,maximum_profit,scheduling_time,time_this_iter_s,training_iteration,timestamp,time_total_s,pid,time_since_restore,timesteps_since_restore,iterations_since_restore
count,80.00000,80.00000,80.00000,80.00000,80.000000,80.000000,80.0,8.000000e+01,80.000000,80.000000,80.000000,80.0,80.0
mean,19.21250,19.21250,48.50000,48.50000,0.088274,0.185620,1.0,1.630711e+09,0.185620,176795.575000,0.185620,0.0,1.0
std,10.38584,10.38584,29.01331,29.01331,0.148942,0.164075,0.0,9.620974e+00,0.164075,1015.235457,0.164075,0.0,0.0
min,1.00000,1.00000,1.00000,1.00000,0.001631,0.024336,1.0,1.630711e+09,0.024336,175313.000000,0.024336,0.0,1.0
25%,9.00000,9.00000,24.75000,24.75000,0.013052,0.080078,1.0,1.630711e+09,0.080078,175939.500000,0.080078,0.0,1.0
50%,17.00000,17.00000,48.50000,48.50000,0.026189,0.141774,1.0,1.630711e+09,0.141774,176686.000000,0.141774,0.0,1.0
75%,30.00000,30.00000,72.25000,72.25000,0.050218,0.208078,1.0,1.630711e+09,0.208078,177666.000000,0.208078,0.0,1.0
max,35.00000,35.00000,96.00000,96.00000,0.816549,0.939815,1.0,1.630711e+09,0.939815,178589.000000,0.939815,0.0,1.0


In [12]:
b

In [68]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2021-09-08_19-00-11")
df = load_ray_experiment(logs)

In [69]:
px.line(df.sort_values("n_tasks"), x="n_tasks", y="n_allocated_tasks", color="scheduler", title="Multiblock DPF killer")

In [66]:
px.line(load_ray_experiment(LOGS_PATH.joinpath("ray/run_and_report_2021-09-10_16-31-04")).sort_values("n_tasks"), x="n_tasks", y="n_allocated_tasks", color="scheduler", title="Single block DPF killer")